Adapted from: https://gist.github.com/MattJBritton/9dc26109acb4dfe17820cf72d82f1e6f

Usage

import ipywidgets as widgets

class MultiCheckboxWidgetI(MultiCheckboxWidget):

    def update(self):

        # implements this method

ui = MultiCheckboxWidgetI(['hello','world'])

display(wid.HBox([ui.multi_select]))

To get the selected options:
selected_options = ui.get_selected_options()

In [ ]:
class MultiCheckboxWidget(object):
    def __init__(self, name, options_list):
        self.name = widgets.HTML(name)
        """ Widget with a search field and lots of checkboxes """
        self.options_dict = {
            x: widgets.Checkbox(
                description=x,
                value=True,
                style={"description_width":"0px"}
            ) for x in options_list
        }

        self.search_widget = widgets.Text()
        self.output_widget = widgets.Output()
        self.options = [x for x in self.options_dict.values()]
        self.options_layout = widgets.Layout(
            overflow='auto',
            border='1px solid black',
            width='300px',
            height='300px',
            flex_flow='column',
            display='flex'
        )

        self.select_all_button = widgets.Button(description='Select All')
        self.select_any_button = widgets.Button(description='Unselect All')
        self.select_widget=widgets.HBox([self.select_any_button, self.select_all_button])
        self.options_widget = widgets.VBox(self.options, layout=self.options_layout)
        self.automatic_update_checkboxe = widgets.Checkbox(
            description="Automatic update",
            value=True,
            style={"description_width":"0px"}
        )
        self.manual_update_button=widgets.Button(description='Update')
        def manual_update(change):
            self.update()
        self.manual_update_button.on_click(manual_update)

        self.automatic_HBox=widgets.HBox([self.automatic_update_checkboxe], layout=widgets.Layout(width='300px'))

        self.multi_select = widgets.VBox([self.name, self.select_widget,
                                          self.search_widget, self.options_widget,
                                          self.automatic_HBox])

        self.automatic_update = True

        def on_select_all_button(change):
            self.automatic_update=False
            for x in self.options:
                x.value=True
            new_options=sorted(
                    [x for x in self.options],
                    key = lambda x: x.description
                )
            self.options_widget.children = new_options
            self.update()
            self.automatic_update=self.automatic_update_checkboxe.value
        self.select_all_button.on_click(on_select_all_button)

        def on_select_automatic_update_checkboxe(change):
            if change['new']:
                self.automatic_update=True
                self.automatic_HBox.children=[self.automatic_update_checkboxe]
                self.update()
            else:
                self.automatic_update=False
                self.automatic_HBox.children=[self.automatic_update_checkboxe, self.manual_update_button]
        self.automatic_update_checkboxe.observe(on_select_automatic_update_checkboxe, names="value")

        def on_select_any_button(change):
            self.automatic_update=False
            for x in self.options:
                x.value=False
            new_options=sorted(
                    [x for x in self.options],
                    key = lambda x: x.description
                )
            self.options_widget.children=new_options
            self.update()
            self.automatic_update=self.automatic_update_checkboxe.value
        self.select_any_button.on_click(on_select_any_button)

        @self.output_widget.capture()
        def on_checkbox_change(change):
            selected_recipe = change["owner"].description
            self.options_widget.children = sorted(
                [x for x in self.options_widget.children],
                key = lambda x: x.value, reverse = True
              )
            if self.automatic_update:
                self.update()

        for checkbox in self.options:
            checkbox.observe(on_checkbox_change, names="value")

        # Wire the search field to the checkboxes
        @self.output_widget.capture()
        def on_text_change(change):
            self.search_input = change['new']
            if self.search_input == '':
                # Reset search field
                new_options = sorted(self.options, key = lambda x: x.value, reverse = True)
            else:
                # Filter by search field using difflib.
                close_matches = [x for x in list(self.options_dict.keys()) if str.lower(self.search_input.strip('')) in str.lower(x)]
                new_options = sorted(
                    [x for x in self.options if x.description in close_matches],
                    key = lambda x: x.value, reverse = True
                )
            self.options_widget.children = new_options

        self.search_widget.observe(on_text_change, names='value')

    def get_selected_options(self):
        return [widget.description for widget in self.options_dict.values() if widget.value]